### This Notebook is a guide on how to run the semi automatic solution. For the 3-D case:

1. Navigate to pre-processing/3D and execute the script that generates the desired geometry. (here we will execute  create_spherical_incl and with a desired porosity) 

```python
#0.3 is the porosity and 0.2, 0.3 are the min and max radius. It is very important to sort the radius of the spheres to be able to archieve higher porosities up to 47%
rad = sorted(get_parameters(0.3,0.2,0.3),reverse=True) 
#these values were chosen in sake of the tutorial it is recommended to use smaller min_r and max_r e.g. 0.09 - 0.12 to archieve high porosities
spheres = create_dataset(len(rad),rad,1.2)
set_physical_regions(rad)
set_resolution(0.07,0.07)
```

2. This will create a text file in meshes/txt_file called spherical_gmsh
```C++
Sphere(2) = {x+0.3549102609084382, y+0.4378583379896717, z+0.6774137001468787, 0.2729716849727446}; 
Sphere(3) = {x+0.6873389809730316, y+0.7235779338235815, z+0.5119612836568483, 0.2549945139986808}; 
Sphere(4) = {x+0.6780590039110005, y+0.29971830545163425, z+0.3222318047364401, 0.25189830672522795}; 
Sphere(5) = {x+0.2144266917278429, y+0.5745433210648649, z+0.23748806384333598, 0.20669214036049305}; 
Sphere(6) = {x+0.7646316740823721, y+0.2227572565595214, z+0.7375795013893071, 0.20628966656176181}; 
v() = BooleanFragments { Volume{1}; Delete; }{ Volume{2 :6}; Delete; }; 
Physical Volume(1) = {7}; 
Physical Volume(2) = {2, 3, 4, 5, 6}; 
Mesh.CharacteristicLengthMin = 0.07; 
Mesh.CharacteristicLengthMax = 0.07; 
```

3. We Navigate to meshes/gmsh and open the empty_3D.geo and transfer the information from spherical gmsh. empty_3D.geo has already defined the cubic domain of length one and the periodicity constraints in the case where the inclusions cross the boundaries. It also defines the physical regions which represents our subdomains.

```C++
SetFactory("OpenCASCADE");
//Starting with a box of length = 1
Box(1) = {0, 0, 0, 1, 1, 1};
x = 0; y = 0; z = 0; 
//-----------------------------------------------------------------------------------------------------------------------
//Copy in the section below the geometric information generated from the pre-processing module in meshes/txt_mesh/
//The geometric information starts from the first line until
//this line -> v() = BooleanFragments { Volume{1}; Delete; }{ Volume{2:n}; Delete; };
//-----------------------------------------------------------------------------------------------------------------------
Sphere(2) = {x+0.3549102609084382, y+0.4378583379896717, z+0.6774137001468787, 0.2729716849727446}; 
Sphere(3) = {x+0.6873389809730316, y+0.7235779338235815, z+0.5119612836568483, 0.2549945139986808}; 
Sphere(4) = {x+0.6780590039110005, y+0.29971830545163425, z+0.3222318047364401, 0.25189830672522795}; 
Sphere(5) = {x+0.2144266917278429, y+0.5745433210648649, z+0.23748806384333598, 0.20669214036049305}; 
Sphere(6) = {x+0.7646316740823721, y+0.2227572565595214, z+0.7375795013893071, 0.20628966656176181}; 
v() = BooleanFragments { Volume{1}; Delete; }{ Volume{2 :6}; Delete; };
//-----------------------------------------------------------------------------------------------------------------------
// STL mesh for bounding boxes:
Geometry.OCCBoundsUseStl = 1;
eps = 1e-3;
//volumes in the bounding box of the original box
vin() = Volume In BoundingBox {-eps,-eps,-eps, 1+eps,1+eps,1+eps};
v() -= vin();
//Deleting the volumes outside the volume of the origina box
Recursive Delete{ Volume{v()}; }
//--------------------------------------------------------------------------------------
//For applying periodic boundaries
//We get all surfaces on the left:
Sxmin() = Surface In BoundingBox{-eps, -eps, -eps, +eps, 1+eps, 1+eps};
For i In {0:#Sxmin()-1}
  // We get the bounding box of the left surfaces
  bb() = BoundingBox Surface { Sxmin(i) };
  // We translate the bounding box to the right and look for surfaces inside i
  Sxmax() = Surface In BoundingBox { bb(0)-eps+1, bb(1)-eps, bb(2)-eps,
                                     bb(3)+eps+1, bb(4)+eps, bb(5)+eps };
  // For all the matches, we compare the corresponding bounding boxes
  For j In {0:#Sxmax()-1}
    bb2() = BoundingBox Surface { Sxmax(j) };
    bb2(0) -= 1;
    bb2(3) -= 1;
    // if they match, we apply the periodicity constraint
    If(Fabs(bb2(0)-bb(0)) < eps && Fabs(bb2(1)-bb(1)) < eps &&
       Fabs(bb2(2)-bb(2)) < eps && Fabs(bb2(3)-bb(3)) < eps &&
       Fabs(bb2(4)-bb(4)) < eps && Fabs(bb2(5)-bb(5)) < eps)
      Periodic Surface {Sxmax(j)} = {Sxmin(i)} Translate {1,0,0};
    EndIf
  EndFor
EndFor

//We get all surfaces on the outside:
Symin() = Surface In BoundingBox{-eps, -eps, -eps, 1+eps, eps, 1+eps};
For i In {0:#Symin()-1}
  // We get the bounding box of each outter surface
  bb() = BoundingBox Surface { Symin(i) };
  // We translate the bounding box to the inside and look for surfaces inside i
  Symax() = Surface In BoundingBox { bb(0)-eps, bb(1)-eps+1, bb(2)-eps,
                                     bb(3)+eps, bb(4)+eps+1, bb(5)+eps };
  // For all the matches, we compare the corresponding bounding boxes
  For j In {0:#Symax()-1}
    bb2() = BoundingBox Surface { Symax(j) };
    bb2(1) -= 1;
    bb2(4) -= 1;
    // if they match, we apply the periodicity constraint
    If(Fabs(bb2(0)-bb(0)) < eps && Fabs(bb2(1)-bb(1)) < eps &&
       Fabs(bb2(2)-bb(2)) < eps && Fabs(bb2(3)-bb(3)) < eps &&
       Fabs(bb2(4)-bb(4)) < eps && Fabs(bb2(5)-bb(5)) < eps)
      Periodic Surface {Symax(j)} = {Symin(i)} Translate {0,1,0};
    EndIf
  EndFor
EndFor

// We get all surfaces on the bottom:
Szmin() = Surface In BoundingBox{-eps, -eps, -eps, 1+eps, 1+eps, +eps};

For i In {0:#Szmin()-1}
  // We get the bounding box of each bottom surface
  bb() = BoundingBox Surface { Szmin(i) };
  // We translate the bounding box to the top and look for surfaces inside i
  Szmax() = Surface In BoundingBox { bb(0)-eps, bb(1)-eps, bb(2)-eps+1,
                                     bb(3)+eps, bb(4)+eps, bb(5)+eps+1 };
  // For all the matches, we compare the corresponding bounding boxes
  For j In {0:#Szmax()-1}
    bb2() = BoundingBox Surface { Szmax(j) };
    bb2(2) -= 1;
    bb2(5) -= 1;
    // ...and if they match, we apply the periodicity constraint
    If(Fabs(bb2(0)-bb(0)) < eps && Fabs(bb2(1)-bb(1)) < eps &&
       Fabs(bb2(2)-bb(2)) < eps && Fabs(bb2(3)-bb(3)) < eps &&
       Fabs(bb2(4)-bb(4)) < eps && Fabs(bb2(5)-bb(5)) < eps)
      Periodic Surface {Szmax(j)} = {Szmin(i)} Translate {0,0,1};
    EndIf
  EndFor
EndFor
//---------------------------------------------------------------------------------------------------------------------------------------
//In this section we copy the rest of the information gathered from the file generated from the pre-processing module in meshes/txt_mesh/
//---------------------------------------------------------------------------------------------------------------------------------------
Physical Volume(1) = {7}; 
Physical Volume(2) = {2, 3, 4, 5, 6}; 
Mesh.CharacteristicLengthMin = 0.07; 
Mesh.CharacteristicLengthMax = 0.07; 
//------------------------------------------------------------------------------------------------------------------------------------------
```
4. Save the geo file, e.g. filename.geo

5. The dolfin/FEniCS and gmsh software need to be installed to generate the 2-D and 3-D meshes. These commands can also be executed from the terminal.

### For 3-D geometries

In [ ]:
#Generates the 3D mesh with name empty_3D
! gmsh -3 filename.geo -format msh2 #replace filename.geo with the actual name of the geo file

In [ ]:
#Converts the mesh into two xml files to be imported in the numerical analysis
! dolfin-convert -i gmsh filename.msh filename.xml #replace filename.geo with the actual name of the geo file

Two xml files are generated in the gmsh directory, filename.xml and filename_physical_region.xml that contain information about the mesh and subdomains.

### For 2D geometires:

In [ ]:
! gmsh -2 filename.geo -format msh2

In [ ]:
! dolfin-convert -i gmsh filename.msh filename.xml

Two xml files are generated in the gmsh directory, filename.xml and filename_physical_region.xml that contain information about the mesh and subdomains.

### For intersected spheres 3-D

Two xml files are generated in the gmsh directory, filename.xml and filename_physical_region.xml that contain information about the mesh and subdomains.

If the option where spheres are intersecting is chosen in create_spherical_incl the tags of the geometries will change 
Here the manual solution used can be shown. It is not the best solution as it consists of manually looking at
the gmsh interface before generating the 3D mesh. What is printed need to be transfered in the geo file.

What we need to do to see the new tags is go into Gmsh graphical software open the geo file and go to tools/visibility/elementary entities and ask to put the elements in ascending order. Search Volume so only the volume elements are shown.

There is possibly a way to do that automatically (https://gmsh.info/doc/texinfo/gmsh.html, A.16)

![gmsh_intersected.png](gmsh_intersected.png)

In [ ]:
v_whole = [10,14,17,19] #all the spheres before the matrix 
v_mat = 20 #matrix volume tag
v_max = 72 # max tag available
for i in range(v_mat+1,v_max+1):
    v_whole.append(i)

print("Physical Volume({}) = {{{}}};".format(1,v_mat))
print("Physical Volume({}) = {{{}}};".format(2,str(v_whole)[1:-1]))

The geo file needs to have this physical information now
```C++
//---------------------------------------------------------------------------------------------------------------------------------------
//In this section we copy the rest of the information gathered from the file generated from the pre-processing module in meshes/txt_mesh/
//---------------------------------------------------------------------------------------------------------------------------------------
Physical Volume(1) = {20};
Physical Volume(2) = {10, 14, 17, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72};
//------------------------------------------------------------------------------------------------------------------------------------------
 
```